# **GIS Lab : Geographical data manipulation in Python**

Antoine Gademer - 2023

**Objectifs** :

The objective of this Lab is to show you how to manipulate geographical data (both vector and raster) in Python in order to :
1. EXPLORE a dataset, discover link and implications
2. PRODUCE a data visualisation in form of an (interactive) map

In order to do that we will try to master several common tasks:

*PART 1 : Vector data*
- Read a vector file (SHP, GPKG, ...), show the attribute table and display the geometry. 
- Filter entities based on the attributes fields
- How to connect to WFS API (remote source of vector data)
- Filter entities based on the geometry field
- Taking care of CRS compatibility
- Display vector data on a map background
- Export modified vector data
- Add important map element : legenda, scale, (north), title/author/date/sources and Export the map in HTML format

*PART 2 : Raster data*
- Read a raster file, show the pixel values and display the corresponding image
- Display vector data on a map background (Taking care of CRS compatibility and reprojection !)

# PART 1 : Vector Data

## - Read a vector file (SHP, GPKG, ...), show the attribute table and display the geometry. 

Vector data are all composed of Point, PolyLines, (Multi-)Polygons. 

But it exists a lot of file format that encode this information differently. The most popular ones are ArcGIS Shape file (.shp), Geopackages (.gpkg) and Geojson (.json) but their is many others (Mapinfo .MIF, etc.)

In return, it also exists several python libraries to read them. [GDAL](https://gdal.org/api/python_bindings.html), [pyshp](https://pypi.org/project/pyshp/), [Geopandas](https://geopandas.org/), [pygis](https://pygis.io), [Leafmap](https://leafmap.org/), [EarthPy](https://earthpy.readthedocs.io/)

In this Notebook, we will use ```Geopandas``` that is at a medium level of abstraction (able to tackle many sources of data, not too specific on the task it can manage).

☞ Download the dataset on Moodle, unzip it and open ```regions-20190101.shp``` with `geopandas`

In [ ]:
#Your code here
%pip install geopandas
import geopandas as gpd

In [ ]:
shp_file_path = "data/Regions/regions-20190101.shp"
gdf = gpd.read_file(shp_file_path)
print(gdf.head())

☞ Use the  `.plot()` function to plot the geometry of the GeoDataframe

In [ ]:
#Your code here
import matplotlib.pyplot as plt
gdf.plot()

☞ Use the `.crs` attribute to show the information about the **Coordinate Reference System** used by the GeoDataframes

In [ ]:
#Your code here
crs_info = gdf.crs
print(crs_info)

☞ Use the `explore()` function on your GeoDataframe to get an interactive map (using the Leaflet library in background)

Explore will try to show all the info contained in the columns as a popup when you click on an entity.


In [ ]:
#Your code here
%pip install folium
%pip install mapclassify
import folium

In [ ]:
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=10)
for idx, row in gdf.iterrows():
    popup = folium.Popup(row.to_string(), max_width=300)
    folium.GeoJson(row.geometry.__geo_interface__, popup=popup).add_to(m)
m.save("interactive_map.html")
gdf.explore()

## - Filter entities based on the attributes fields

☞ Select the Region Bretagne and show it.

<details>

<summary>Tips</summary>

GeoDataframe are basically pandas's Dataframe + a geometry field. You can use the filtering capacities of pandas.
```
df[df["myColumn"]=="Value"] # Select only the rows where the myColumn field equal "Value".
```

</details>

In [ ]:
#Your code here
bretagne = gdf[gdf['nom'] == 'Bretagne']
bretagne.plot()

In [ ]:
bretagne.explore()

## - How to connect to WFS API (remote source of vector data)

Geographical Information System have a long history of accessing remote data via APIs. Since 2003, the WFS (Web Feature Service) define an interface protocol to query remote server and accesss vector data.

The French National Geographic Institute (IGN) propose a lot of services freely avaiable (More info : https://geoservices.ign.fr/services-web-experts)

Example:
```
https://wxs.ign.fr/topographie/geoportail/wfs?SERVICE=WFS&VERSION=2.0.0&request=GetFeature&OUTPUTFORMAT=application/json&typename=BDTOPO_V3:cours_d_eau&CQL_FILTER=toponyme%20ilike%20%27la%20Loire%27
```

- `topographie` is the keyword to access the BD TOPO (a collection of vector data on many subject : administrative, rivers, roads, etc.)
- `typename=BDTOPO_V3:cours_d_eau` we ask for the `cours_d_eau` (i.e. water stream) layer.
- `CQL_FILTER=toponyme ilike "la Loire"` we add a filter (i.e. a WHERE condition) on the `toponyme` property. Here we select all the water stream named `la Loire`.

Notes : 
1. the service will send 1000 rows AT MOST
2. the `&count=XX` option can limit the number of row returned to XX max.
3. if you want to use a joker, you can write `CQL_FILTER=toponyme ilike "%Loire%"` (but think to encode it as URL --> `CQL_FILTER=toponyme%20ilike%20%27%25Loire%25%27`. '%25' is the [url encoding](https://www.w3schools.com/tags/ref_urlencode.ASP) of the '%' character).
4. you can combine filters with the `and` operator


**Important note**: Geopandas can read WFS urls with the `read_file()` function 😊

☞ Use the previously described url to load a GeoDataframe containing all the water stream called "la Loire".

Display the content of the GeoDataframe

In [ ]:
#Your code here
wfs_url = "https://wxs.ign.fr/topographie/geoportail/wfs?SERVICE=WFS&VERSION=2.0.0&request=GetFeature&OUTPUTFORMAT=application/json&typename=BDTOPO_V3:cours_d_eau&CQL_FILTER=toponyme%20ilike%20%27la%20Loire%27"
gdf_loire = gpd.read_file(wfs_url)
print(gdf_loire)
gdf_loire.plot()

<details>

<summary>Help it says "Cannot convert Timestamp to JSON when I want to use the `explore()` function!</summary>

Behind the curtain, `explore()` is converting your data in GeoJson and the Timestamp type is not convertible.

Just skip the problematic columns:
```
gdf_river.loc[:,~gdf_river.columns.isin(['date_creation', 'date_modification'])].explore()
```

</details>

There seem to be usurpers (or more likely homonyms) in the list.

The `length` attribute of the GeoDataframe calculate the length of the polyline of each "geometry" field.

Important note: the WGS84 CRS gives coordinates in degree. It is not a good projection for geometrical measurement.
Geopandas allows you to convert your geometry to a new CRS with the `to_crs()` function.

☞ 
- Convert the dataframe to the CRS `EPSG:2154` (Lambert93).
- Use the `crs` attribute to see the unit of the coordinates.
- Calculate the length **in kilometer** of each river
- Save the information as a new field "Length_in_kilometer" in the original GeoDataframe
- Print the values

Question: Which one is the "right" one ? (Considering https://en.wikipedia.org/wiki/Loire )

In [ ]:
#Your code here
#Your code here
gdf_loire = gdf_loire.to_crs(epsg=2154)

print(gdf_loire.crs)

gdf_loire['Length_in_kilometer'] = gdf_loire['geometry'].length / 1000

print(gdf_loire[['toponyme', 'Length_in_kilometer']])


### - Filter entities based on the geometry field

The `intersects` function can be used to filter entities that intersects **a geometrical object**. It return a Series of True/False value based on the fact that each Geometry of the Dataframe intersect of not the geometrical object.

*Warning: If you give it two GeoSeries, it will try to test row-by-row (not each row with all the row of the other).*

The geometrical object associated to a row is in the `"geometry"` field.

☞ 
- Extract the geometry of the Loire river
- Filter the regions that intersects the Loire river and display them.

In [ ]:
#Your code here
gdf_loire = gpd.read_file(wfs_url)
gdf_loire = gdf_loire[gdf_loire.index==2]
loire_geometry = gdf_loire.unary_union
intersected_regions = gdf[gdf["geometry"].intersects(loire_geometry)]

print(intersected_regions)
intersected_regions.plot()

## - Taking care of CRS compatibility

The `BD_ROUTE_roads.gpkg` is a vector data file from another of IGN product : the BD ROUTE.

☞
- Load the data with Geopandas
- Select the roads that interesects at least one of the previously selected regions

Tips: 
- `unary_union` attribute allows to get the union of all geometry of your GeoDataframe (try a `display()` on it to understand what you have)
- `simplify(tolerance=0.1)` would allow you to get a simpler geometry (and a MUCH quicker computation time on the `intersects()`)

**⚠** tolerance is in the same unit as the CRS your are using. For WGS84, 0.1° is a good compromise but for Lambert93, 0.1m is not much 😛. Don't hesitate to take a much higher value (5km per ex.)

**A good practice is to display the geometry your using before calling `intersects()`**

<details>

<summary>Tips</summary>

Have you check the CRS of the two layers your try to intersect ?

Remember `.crs` and `.to_crs()`
</details>

In [ ]:
#Your code here
region_geometry = intersected_regions.unary_union.simplify(tolerance=5000)
region_geometry

In [ ]:
roads_gdf = gpd.read_file("data/BD_ROUTE/BD_ROUTE_roads.gpkg")
roads_gdf = roads_gdf.to_crs(gdf.crs)
intersected_roads = roads_gdf[roads_gdf.intersects(region_geometry)]
intersected_roads

## - Display vector data (on a map background)

### With matplotlib

Your mastery of matplotlib should allow you to combine all the "plot"

☞ Plot the selected regions,river,roads together

- The regions should be grey and only show the boundary (no filling)
- The river should be blue
- The roads should be brown
- Highways ('Autoroute') should be 2pt width and the Nationale roads ('Nationale') 1pt width (look at the "CLASS_ADM" field and the `linewidth` plot property)
- It should show a legend

Tips: the `boundary` attribute of a GeoDataframe return a **polyline** corresponding to the boundary (instead of the original polygon)

![The resulting image](./img/final_plot_example.png)

<details>

<summary>Help : how to combine the plots?</summary>

In general, `plot()` allows you pass the axes (`ax=`) where you want to plot !
</details>

<details>

<summary>Help : how do I create a legend?</summary>

The `plt.legend()` create the legend of the plot. But you need to have specified the `label` attribute in each of you plots !
</details>

<details>

<summary>Help : how do I differentiate the linewidth based on attribute?</summary>

`plot()` is able to do categorical colors by itself (see the `column=` and `cmap=` parameters). But you wont be able to control a lot of parameter (including the `linewidth`).

A smart way is to us the `groupBy()` function of Pandas and to plot separatly each group.
```
linewidthMap = {'Autoroute':2,'Nationale':1} # A dict to store the relation between class and linewidth
labelMap = {'Autoroute':'Highways','Nationale':'National'} # A dict to store the relation between class and label
# You could have other for color, linestyle, etc.
for ctype, data in gdf_roads_intersect.groupby('CLASS_ADM'): # Group by CLASS_ADM
    data.plot(ax=ax,
              color='brown',label=labelMap[ctype],linewidth=linewidthMap[ctype])
```
</details>

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
#Your code here
intersected_regions.boundary.plot(ax=ax, color='grey', label='Regions')
gdf_loire.plot(ax=ax, color='blue', label='La Loire')
linewidthMap = {'Autoroute': 2, 'Nationale': 1}
labelMap = {'Autoroute': 'Highways', 'Nationale': 'National Roads'}

for ctype, data in intersected_roads.groupby('CLASS_ADM'):
    data.plot(ax=ax, color='brown', label=labelMap[ctype], linewidth=linewidthMap[ctype])
ax.legend()
plt.show()

The results is not bad, but it is not that easy to localize the information.

Having a map background (like G..gle Maps or OpenStreetMaps (OSM)) would be a great addition.

Having the possibility to pan/zoom around would also be great!

### With Folium

Leaflet is the javascript engine behind the nice `explore()` function.

To combine it the way we have with `plot()`, we need a map object created using `folium` a python binding around Leaflet.

In [ ]:
import folium

map = folium.Map() # Create the map object

map.fit_bounds([[41.333191,-5.1412766] ,  [51.0889911,9.560053]]) # Zooming to France bounding box

display(map) # Actually displaying the map

Ok. We have our map. How can we add our vector layers?

You can pass the `map` object to your `explore()` function (the parameter is called `m=`).
```
gdf.explore(m=map)
```

**⚠ Remember that leaflet only works with data in WGS84/EPSG:4326 CRS**

You can find all the parameter of the `explore()` function here : https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html

We can point you to several interesting ones:
- `color=`
- `tooltip=`
- `popup=`
- `style_kwds=` and in particular : `style_kwds={'fill':False}` and the very powerful : 
```
style_kwds={'style_function':lambda x: {
    "fillColor": "#0000ff"
    if x["properties"]["nom"] == "Pays de la Loire"
    else "#00ff00"
}}
```
*(You can do practically **any** customization with this lambda function)*

☞ Plot on a Folium map the selected regions,river,roads together

- The river should be blue
- The roads should be brown with the Highways 2pt and National 1pt width (Use `style_function` power or the `groupby` technique)
- The regions should be grey and only show the boundary (no filling)
- It should show a legend

![The result we want to obtain](./img/final_folium_example.png)

In [ ]:
#Your code here
intersected_regions= intersected_regions.to_crs("EPSG:4326")
gdf_loire = gdf_loire.to_crs("EPSG:4326")
if 'date_creation' in gdf_loire.columns or 'date_modification' in gdf_loire.columns:
    gdf_loire = gdf_loire.drop(columns=['date_creation', 'date_modification'])
intersected_roads = intersected_roads.to_crs("EPSG:4326")
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)
folium.GeoJson(gdf_loire, name='Loire', style_function=lambda x: {'color': 'blue'}).add_to(m)
for ctype, data in intersected_roads.groupby('CLASS_ADM'):
    style = {'color': 'brown', 'weight': 2 if ctype == 'Autoroute' else 1}
    folium.GeoJson(data, name=f'Roads - {ctype}', style_function=lambda x, style=style: style).add_to(m)
folium.GeoJson(intersected_regions, name='Regions Boundary', style_function=lambda x: {'color': 'gray', 'fillOpacity': 0}).add_to(m)
folium.LayerControl().add_to(m)
display(m)

<details>

<summary>Help it says "Cannot convert Timestamp to JSON!</summary>

Timestamp type is not convertible to (Geo)Json.

Just skip the problematic columns:
```
gdf_river.loc[:,~gdf_river.columns.isin(['date_creation', 'date_modification'])].explore()
```
or in reverse, choose the pertinent column to show (don't forget the `'geometry'` field!):
```
gdf_river[['toponyme','Length_in_kilometer','geometry']].explore()
```

</details>

<details>

<summary>Help : How do I add a legend?</summary>

`folium` has a `LayerControl` object for you 😊

Just skip the problematic columns:
```
map.add_child(folium.LayerControl(collapsed=False))
```

But your GeoJson must have a `name=` attribute set to serve as labels.

</details>

## - Export modified vector data

We you want to save your work and your modified GeoDataframe, it is as simple as calling the `to_file()` function and choosing a vector data file extension : '.gpkg' (counseled) or '.shp' (*remember that you need to send/save all the other files : shx,dbf,etc. along with the shp*)

☞ Save your modified GeoDataframes into three GPKG files

In [ ]:
#Your code here
gdf_loire.to_file('loire.gpkg', driver='GPKG')
intersected_regions.to_file('intersected_regions.gpkg', driver='GPKG')
intersected_roads.to_file('intersected_roads.gpkg', driver='GPKG')

## - Add important map element : legenda, scale, (north), title/author/date/sources and Export the map in HTML format

To be a **valid** map, a map should present a minimum number of information that should **always** be there :
- A Title (pertinent in regard of the **goal** of the map)
- A legenda
- A scale
- The author name
- The date of creation
- The sources (and eventually the date of sources)
- A North arrow (if the North is not Up)

To add a **scale** to your map, simply use the `control_scale=True` parameter when creating the map.
```
map = folium.Map(control_scale = True)
```

To add a textual element like you need to add HTML blocks to your map with the `folium.Element` object.

The `addFixedOverlay()` function will help you if you're not a CSS guru 😛

In [ ]:
def addFixedOverlay(map, txtInHTML,cornerRef="BL",position=(50,50),z_index=9999,div_style=""):
    switch = {"BL":'bottom: {}px; left:{}px;',
               "TL":'top: {}px; left:{}px;',
               "TR":'top: {}px; right:{}px;',
               "BR":'bottom: {}px; right:{}px;',
               "TC":'top: {}px; left:50%;margin-left:{}px;', # second param should be the half of the width
               "BC":'bottom: {}px; left:50%;margin-left:{}px;', # second param should be the half of the width
             }
    
    print(switch[cornerRef].format(position[0],position[1]))
    overlay_html="""
<div style="position: fixed; {} z-index: {}; background-color: white; padding: 10px; border: 2px solid grey; {} ">
{}
</div>
""".format(switch[cornerRef].format(position[0],position[1]),z_index,div_style,txtInHTML)
    map.get_root().html.add_child(folium.Element(overlay_html))


Example:
```
addFixedOverlay(map,"<p>Author: A. Gademer. Date: 26/09/2023.</p><p>Sources : IGN&#39s BD TOPO v3 (WFS), IGN&#39s BD ROUTE 2021, OSM Regions shapes</p>",cornerRef="BR",position=(15,10),div_style="max-width:300px;")
```

To add have a pretty **legend**, we can add a `folium.LayerControl()` to your map, simply use the `control_scale=True` parameter when creating the map.
```
map.add_child(folium.LayerControl(collapsed=False)) # You need to call it AFTER adding all your layers.
```


![Ugly legend](./img/ugly_legend.png)

<details>

<summary>Help it my legend looks ugly</summary>

You should set the `name=` parameter of the `explore()` function.

Note that you can pass any HTML code, so you can do pretty legend instead !

Example:
```
gdf_regions.explore(m=map,color="grey",name='<span style="color: grey;">▬</span> Regions (Boundary)')
```

</details>

**Export**: To export the folium map just use:
```
map.save("MYNAME_PART1_map.html")
```

But don't forget to check is there is some depencies (images, data) to zip together in the archive you deposit.


☞ Add a *pertinent* title and a scale to your map (and any other element you achieve) and export your map in HTML.

In [ ]:
# Your code here
intersected_regions = intersected_regions.to_crs("EPSG:4326")
gdf_loire = gdf_loire.to_crs("EPSG:4326")
if 'date_creation' in gdf_loire.columns or 'date_modification' in gdf_loire.columns:
    gdf_loire = gdf_loire.drop(columns=['date_creation', 'date_modification'])
intersected_roads = intersected_roads.to_crs("EPSG:4326")
m = folium.Map(location=[46.603354, 1.888334],
                zoom_start=6, control_scale=True)
title_txt = "Region contact with loire"
title_html = '''
            <h3 align="center" style="font-size:16px"><b>{}</b></h3>
            '''.format(title_txt)

m.get_root().html.add_child(folium.Element(title_html))
author_info = "<p>Author: W. Haoyu. Date: 29/09/2023.</p><p>Sources : IGN&#39s BD TOPO v3 (WFS), IGN&#39s BD ROUTE 2021, OSM Regions shapes</p>"
addFixedOverlay(m, author_info, cornerRef="BR", position=(
    15, 10), div_style="max-width:300px;")
folium.GeoJson(gdf_loire, name='Loire', style_function=lambda x: {
                'color': 'blue'}).add_to(m)
for ctype, data in intersected_roads.groupby('CLASS_ADM'):
    style = {'color': 'brown', 'weight': 2 if ctype == 'Autoroute' else 1}
    folium.GeoJson(
        data, name=f'Roads - {ctype}', style_function=lambda x, style=style: style).add_to(m)
folium.GeoJson(intersected_regions, name='Regions Boundary',
                style_function=lambda x: {'color': 'gray', 'fillOpacity': 0}).add_to(m)
m.add_child(folium.LayerControl(collapsed=False))
m.save("MyMap.html")
display(m)


# PART 2 : Raster Data

## - Read a raster file, show the pixel values and display the corresponding image

`geopandas` is perfect for vector data, but it does not know how to manage raster data.

It exists several library for managing raster data : [GDAL](https://gdal.org/api/python_bindings.html), [rasterio](https://rasterio.readthedocs.io), [rioxarray](https://corteva.github.io/rioxarray/), [pygis](https://pygis.io), [Leafmap](https://leafmap.org/), [EarthPy](https://earthpy.readthedocs.io/)

In this lab, we will use `rioxarray` that seem to me a good compromise in my tests.

The main difference between classical image (let say jpg/png/tif) and raster (let say (geo)tif or jp2) is the presence of CRS information.

To **open** a raster file we will use the `rioxarray.open_rasterio(filename)` function.

You can see the **CRS** of the file with `raster.rio.crs`

You can see the **dimensions** of the raster with `raster.shape`

You can have the **bounding box** of the raster with `raster.rio.bounds()`

You can have more information of the object with `display(raster)`

To **plot** the content of the raster, we need to use the `show` function imported from rasterio **on the `raster.data` attribute**:
```
from rasterio.plot import show
show(raster.data)
```

☞ Open the `La_chapelle_Montinard.tif` raster image and show all the important information (and plot it).

In [ ]:
%pip install rioxarray
import rioxarray
from rasterio.plot import show

In [ ]:
#Your code here
raster = rioxarray.open_rasterio('data/BD_ORTHO/La_chapelle_Montinard.tif')
raster.rio.crs
raster.shape
raster.rio.bounds()
show(raster.data)
display(raster)

You could also use Pillow, but beware that the order of the dimensions are reversed.
Pillow want Height, Width, Channels when rioxarray is Channel, Height, Width
```
from PIL import Image
display(Image.fromarray(raster.data.transpose([1,2,0]))) # Height, Width, Channels (rioxarray is Channel, Height, Width)
```

In [ ]:
#Your code here
from PIL import Image
image_data_pillow = raster.data.transpose([1, 2, 0])
image_pillow = Image.fromarray(image_data_pillow)
display(image_pillow)

Side note: The IGN's BD ORTHO offer a 20cm GSD (Ground Sample Distance), i.e 1 pixel = 20cm, a quite neat precision, all over the territory !

## - Display vector data on a map background (Taking care of CRS compatibility and reprojection !)

Finally, we may want to show it in our Folium map !

On the principle it is quite simple:
```
map.add_child(folium.raster_layers.ImageOverlay(
        name="Raster image",
        image=image_for_folium,
        bounds=bounds_for_folium,
                ))
```

but.... 
- the image array should be in Height, Width, Channels (like Pillow)
- we need to have the bounds coordinates of the image... in [[LonMin, LatMin],[LonMax, LatMax]] format

Note : Lon/Lat are the  WSG84/EPSG:4326 coordinates. Check your CRS.

### A little detour: Converting coordinates with pyproj

pyproj is the python library that geopandas is using to manage the CRS. We can use it directly here to convert our EPSG:2154 to EPSG:4326.

In [ ]:
epsg_bounds=raster.rio.bounds() # The bounding box of the raster

from pyproj import Transformer
transformer = Transformer.from_crs(raster.rio.crs, "EPSG:4326") # We create a transformer object

# And we use it to transform our pair of coordinates.
wgs_coord = [ transformer.transform(epsg_bounds[0],epsg_bounds[1]) , transformer.transform(epsg_bounds[2],epsg_bounds[3])  ]

#Note that in the end, we should have [[LonMin, LatMin],[LonMax, LatMax]]
display(wgs_coord)

☞ Now, you should have everything you need to add the raster image to your folium map

In [ ]:
#Your code here
import tempfile
temp_image_path = tempfile.mktemp(suffix=".png")
image_pillow.save(temp_image_path, format="PNG")

folium.raster_layers.ImageOverlay(
    name="Raster image",
    image=temp_image_path,
    bounds=wgs_coord
).add_to(m)
display(m)

Questions : 
- What can you sau of the superimposition of the image and the map background?
- Does the image have straight angles? Why?

It will stack perfectly if the location information is the same.
Yes, the image contains right angles. This is because it is a vector graphic representing a building floor plan where straight lines and right angles are critical to accurately depicting the building layout.

☞ Combine the three previously created vector layers with the raster layer (The raster should be at the bottom).

- The map should have all the elements (Title/Legend/etc.)
- The map should be zoom to the surroundings of the raster image
- The vector should follow the style previously define (grey, blue, brown with various weight, etc.)

Save it as an HTML file.

In [ ]:
# Your code here
m = folium.Map(location=[(wgs_coord[0][0] + wgs_coord[1][0]) / 2, (wgs_coord[0][1] + wgs_coord[1][1]) / 2],
                zoom_start=16)

folium.GeoJson(intersected_regions, name='Regions Boundary',
                style_function=lambda x: {'color': 'gray', 'fillOpacity': 0}).add_to(m)

folium.GeoJson(gdf_loire, name='Loire', style_function=lambda x: {'color': 'blue'}).add_to(m)

for ctype, data in intersected_roads.groupby('CLASS_ADM'):
    style = {'color': 'brown', 'weight': 2 if ctype == 'Autoroute' else 1}
    folium.GeoJson(data, name=f'Roads - {ctype}',
                    style_function=lambda x, style=style: style).add_to(m)
folium.raster_layers.ImageOverlay(
    name="Raster image",
    image=temp_image_path,
    bounds=wgs_coord
).add_to(m)
title_txt = "Map Title"
title_html = f'<h3 align="center" style="font-size:16px"><b>{title_txt}</b></h3>'
m.get_root().html.add_child(folium.Element(title_html))

author_info = "<p>Author: Wang Haoyu. Date: 29/09/2023.</p>"
addFixedOverlay(m, author_info, cornerRef="BR", position=(15, 10), div_style="max-width:300px;")

m.add_child(folium.LayerControl(collapsed=False))
m.save("combined_map.html")
display(m)

# Your deposit will be a zip archive containing your functional HTML map + this notebook **cleared from all cell outputs** (for size consideration).